# Module Optimisation

In [1]:
import dspy
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
import os
from getpass import getpass

# Configure the language model
#if "OPENAI_API_KEY" not in os.environ:
#    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key: ")
#lm = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=4096)
lm = dspy.LM("groq/meta-llama/llama-4-scout-17b-16e-instruct", api_key="os.environ/GROQ_API_KEY")
dspy.configure(lm=lm)

# Alternative: Use a local model like Ollama
# lm = dspy.OllamaLocal(model='llama3.2', max_tokens=4096)
# dspy.settings.configure(lm=lm)

In [2]:
# Synthetic dataset for financial news sentiment analysis
trainset = [
    dspy.Example(
        headline="TechCorp reports record-breaking quarterly earnings, exceeding analyst expectations.",
        answer="Up"
    ).with_inputs("headline"),
    dspy.Example(
        headline="TechCorp faces regulatory fines due to compliance issues in overseas markets.",
        answer="Down"
    ).with_inputs("headline"),
    dspy.Example(
        headline="TechCorp announces new product launch, expected to capture significant market share.",
        answer="Up"
    ).with_inputs("headline"),
    dspy.Example(
        headline="TechCorp's CEO resigns amid internal restructuring, no clear successor named.",
        answer="Down"
    ).with_inputs("headline"),
    dspy.Example(
        headline="TechCorp stock trading volume steady, no major news reported this week.",
        answer="Neutral"
    ).with_inputs("headline"),
]

devset = [
    dspy.Example(
        headline="TechCorp secures major government contract for AI solutions.",
        answer="Up"
    ).with_inputs("headline"),
    dspy.Example(
        headline="TechCorp reports supply chain disruptions impacting production.",
        answer="Down"
    ).with_inputs("headline"),
    dspy.Example(
        headline="TechCorp maintains stable dividend payouts with no changes announced.",
        answer="Neutral"
    ).with_inputs("headline"),
]

In [3]:
# Define the DSPy signature for stock price movement prediction
class StockPricePrediction(dspy.Signature):
    """Predict whether a stock's price will go up, down, or remain neutral based on a news headline."""
    headline = dspy.InputField(desc="A financial news headline about a company.")
    answer = dspy.OutputField(desc="Predicted stock price movement: Up, Down, or Neutral.")

# Define the DSPy module with Chain of Thought
class StockPriceCoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(StockPricePrediction)

    def forward(self, headline):
        return self.prog(headline=headline)

# Define a simple evaluation metric
def stock_prediction_metric(example, pred, trace=None):
    return example.answer.lower() == pred.answer.lower()

# Initialize and compile the module with BootstrapFewShotWithRandomSearch
config = dict(
    max_bootstrapped_demos=4,
    max_labeled_demos=4,
    num_candidate_programs=8,
    num_threads=4
)
optimizer = BootstrapFewShotWithRandomSearch(metric=stock_prediction_metric, **config)
compiled_cot = optimizer.compile(StockPriceCoT(), trainset=trainset, valset=devset)

# Save the optimized module
compiled_cot.save('stock_price_cot.json')

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 8 candidate sets.
Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:00<00:00,  3.18it/s]

2025/07/19 04:16:43 INFO dspy.evaluate.evaluate: Average Metric: 3 / 3 (100.0%)



New best score: 100.0 for seed -3
Scores so far: [100.0]
Best score so far: 100.0
Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:00<00:00,  6.26it/s]

2025/07/19 04:16:44 INFO dspy.evaluate.evaluate: Average Metric: 3 / 3 (100.0%)



Scores so far: [100.0, 100.0]
Best score so far: 100.0


...

Average Metric: 1.00 / 1 (100.0%):  33%|███▎      | 1/3 [00:03<00:07,  3.55s/it]

2025/07/19 04:18:01 ERROR dspy.utils.parallelizer: Error for Example({'headline': 'TechCorp secures major government contract for AI solutions.', 'answer': 'Up'}) (input_keys={'headline'}): Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `meta-llama/llama-4-scout-17b-16e-instruct` in organization `org_01jynyvq87f3may0sgyx33aha2` service tier `on_demand` on requests per minute (RPM): Limit 30, Used 30, Requested 1. Please try again in 1.977s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"requests","code":"rate_limit_exceeded"}}
. Set `provide_traceback=True` for traceback.


Average Metric: 2.00 / 2 (100.0%): 100%|██████████| 3/3 [00:07<00:00,  2.40s/it]

2025/07/19 04:18:01 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)



Scores so far: [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 66.67, 66.67, 66.67, 66.67]
Best score so far: 100.0
11 candidate programs found.


In [4]:
# Example usage: Predict stock price movement for a new headline
new_headline = "TechCorp announces breakthrough in renewable energy technology."
prediction = compiled_cot(headline=new_headline)
print(f"Headline: {new_headline}")
print(f"Predicted stock price movement: {prediction.answer}")

# Evaluate on devset
correct = 0
for example in devset:
    pred = compiled_cot(headline=example.headline)
    if stock_prediction_metric(example, pred):
        correct += 1
accuracy = correct / len(devset)
print(f"Accuracy on devset: {accuracy:.2%}")

Headline: TechCorp announces breakthrough in renewable energy technology.
Predicted stock price movement: Up
Accuracy on devset: 100.00%


# Prompt Optimisation

In [5]:
import dspy
from dspy.teleprompt import MIPROv2
from dspy.evaluate import Evaluate
import os
from getpass import getpass

# Configure the language model (example uses OpenAI; adjust for your setup)
#if "OPENAI_API_KEY" not in os.environ:
#    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key: ")
#lm = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=4096)
#dspy.settings.configure(lm=lm)

# Alternative: Use a local model like Ollama
# lm = dspy.OllamaLocal(model='llama3.2', max_tokens=4096)
# dspy.settings.configure(lm=lm)

In [6]:
# Synthetic dataset for portfolio optimization
trainset = [
    dspy.Example(
        headline="TechCorp Q2 earnings soar 25%, beating estimates; AI division grows 40%.",
        fundamentals="P/E: 22, Rev Growth: 15%",
        market_conditions="VIX: 18 (moderate)",
        answer="Buy",
        rationale="Positive earnings and strong AI growth outweigh moderate market volatility."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="PharmaCo faces patent expiration; new drug trials delayed.",
        fundamentals="P/E: 30, Rev Growth: -5%",
        market_conditions="VIX: 25 (high)",
        answer="Sell",
        rationale="Patent issues and negative growth combined with high volatility suggest risk."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="EnergyCo secures $1B renewable contract; stock steady.",
        fundamentals="P/E: 18, Rev Growth: 10%",
        market_conditions="VIX: 15 (low)",
        answer="Buy",
        rationale="Contract win and solid fundamentals in stable market favor upside."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="RetailCo reports flat sales amid consumer slowdown.",
        fundamentals="P/E: 20, Rev Growth: 0%",
        market_conditions="VIX: 20 (moderate)",
        answer="Hold",
        rationale="Flat performance and moderate market conditions suggest no immediate action."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="AutoCo launches EV model, but supply chain issues persist.",
        fundamentals="P/E: 25, Rev Growth: 8%",
        market_conditions="VIX: 22 (moderate)",
        answer="Hold",
        rationale="EV launch is positive, but supply chain risks and moderate volatility balance out."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="BankCo increases dividends after strong loan growth.",
        fundamentals="P/E: 15, Rev Growth: 12%",
        market_conditions="VIX: 16 (low)",
        answer="Buy",
        rationale="Strong fundamentals and low volatility support a positive outlook."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="TechCorp rival gains market share with new AI product.",
        fundamentals="P/E: 28, Rev Growth: 5%",
        market_conditions="VIX: 20 (moderate)",
        answer="Sell",
        rationale="Competitive pressure and modest growth in volatile market suggest downside."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="PharmaCo secures FDA approval for new drug.",
        fundamentals="P/E: 22, Rev Growth: 10%",
        market_conditions="VIX: Institutions 17 (low)",
        answer="Buy",
        rationale="FDA approval boosts growth potential in a stable market."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="EnergyCo faces regulatory probe over emissions.",
        fundamentals="P/E: 20, Rev Growth: -2%",
        market_conditions="VIX: 24 (high)",
        answer="Sell",
        rationale="Regulatory risks and negative growth in volatile market are concerning."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="RetailCo expands e-commerce platform successfully.",
        fundamentals="P/E: 18, Rev Growth: 12%",
        market_conditions="VIX: 19 (moderate)",
        answer="Buy",
        rationale="E-commerce growth and solid fundamentals outweigh moderate volatility."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="AutoCo reports strong quarterly sales despite chip shortage.",
        fundamentals="P/E: 20, Rev Growth: 10%",
        market_conditions="VIX: 21 (moderate)",
        answer="Buy",
        rationale="Strong sales performance mitigates supply chain concerns."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="BankCo faces cybersecurity breach, stock dips.",
        fundamentals="P/E: 22, Rev Growth: 5%",
        market_conditions="VIX: 23 (high)",
        answer="Sell",
        rationale="Cybersecurity issues and high volatility pose significant risks."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="TechCorp announces share buyback program.",
        fundamentals="P/E: 24, Rev Growth: 8%",
        market_conditions="VIX: 16 (low)",
        answer="Buy",
        rationale="Buyback signals confidence; strong fundamentals in stable market."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="PharmaCo reports mixed trial results for new vaccine.",
        fundamentals="P/E: 25, Rev Growth: 3%",
        market_conditions="VIX: 20 (moderate)",
        answer="Hold",
        rationale="Mixed results balance growth potential and market uncertainty."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="EnergyCo divests non-core assets to focus on renewables.",
        fundamentals="P/E: 19, Rev Growth: 7%",
        market_conditions="VIX: 18 (moderate)",
        answer="Buy",
        rationale="Strategic focus on renewables is positive in stable market."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="RetailCo faces labor strikes, impacting operations.",
        fundamentals="P/E: 23, Rev Growth: -3%",
        market_conditions="VIX: 22 (moderate)",
        answer="Sell",
        rationale="Operational disruptions and negative growth suggest caution."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="AutoCo secures government EV subsidies.",
        fundamentals="P/E: 21, Rev Growth: 9%",
        market_conditions="VIX: 17 (low)",
        answer="Buy",
        rationale="Subsidies enhance growth in a stable market."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="BankCo reports stable earnings, no major changes.",
        fundamentals="P/E: 20, Rev Growth: 4%",
        market_conditions="VIX: 19 (moderate)",
        answer="Hold",
        rationale="Stable performance but no strong catalyst for action."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="TechCorp hit by global chip shortage, production slows.",
        fundamentals="P/E: 27, Rev Growth: 2%",
        market_conditions="VIX: 23 (high)",
        answer="Sell",
        rationale="Supply chain issues and high volatility outweigh modest growth."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="PharmaCo partners with global health organization.",
        fundamentals="P/E: 21, Rev Growth: 11%",
        market_conditions="VIX: 16 (low)",
        answer="Buy",
        rationale="Partnership boosts growth potential in stable market."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
]

devset = [
    dspy.Example(
        headline="TechCorp unveils new AI chip, analysts optimistic.",
        fundamentals="P/E: 23, Rev Growth: 14%",
        market_conditions="VIX: 17 (low)",
        answer="Buy",
        rationale="Innovative product and strong fundamentals in stable market."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="PharmaCo faces lawsuit over drug side effects.",
        fundamentals="P/E: 28, Rev Growth: -4%",
        market_conditions="VIX: 24 (high)",
        answer="Sell",
        rationale="Legal risks and negative growth in volatile market."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="EnergyCo reports steady output, no new contracts.",
        fundamentals="P/E: 19, Rev Growth: 5%",
        market_conditions="VIX: 20 (moderate)",
        answer="Hold",
        rationale="Stable performance but no strong growth catalyst."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="RetailCo launches loyalty program, mixed reviews.",
        fundamentals="P/E: 21, Rev Growth: 6%",
        market_conditions="VIX: 19 (moderate)",
        answer="Hold",
        rationale="Loyalty program impact unclear, moderate market conditions."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="AutoCo ramps up EV production with new factory.",
        fundamentals="P/E: 20, Rev Growth: 12%",
        market_conditions="VIX: 16 (low)",
        answer="Buy",
        rationale="Increased production capacity in stable market is positive."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="BankCo reports loan defaults rising slightly.",
        fundamentals="P/E: 22, Rev Growth: 3%",
        market_conditions="VIX: 21 (moderate)",
        answer="Sell",
        rationale="Rising defaults and modest growth suggest caution."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="TechCorp expands cloud services, strong demand expected.",
        fundamentals="P/E: 24, Rev Growth: 10%",
        market_conditions="VIX: 18 (moderate)",
        answer="Buy",
        rationale="Cloud expansion aligns with demand in stable market."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="PharmaCo halts trials due to funding issues.",
        fundamentals="P/E: 26, Rev Growth: -2%",
        market_conditions="VIX: 23 (high)",
        answer="Sell",
        rationale="Funding issues and high volatility pose significant risks."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="EnergyCo signs new solar project deal.",
        fundamentals="P/E: 18, Rev Growth: 8%",
        market_conditions="VIX: 17 (low)",
        answer="Buy",
        rationale="New deal and solid fundamentals in stable market."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="RetailCo reports stable holiday sales.",
        fundamentals="P/E: 20, Rev Growth: 4%",
        market_conditions="VIX: 19 (moderate)",
        answer="Hold",
        rationale="Stable sales but no significant growth driver."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
]

In [10]:
# Define the DSPy signature for portfolio optimization
class PortfolioRecommendation(dspy.Signature):
    """Given a news headline, company fundamentals, and market conditions, recommend whether to Buy, Hold, or Sell a stock, and provide a rationale."""
    headline = dspy.InputField(desc="A financial news headline about a company.")
    fundamentals = dspy.InputField(desc="Company fundamentals, e.g., P/E ratio, revenue growth.")
    market_conditions = dspy.InputField(desc="Market conditions, e.g., VIX level and interpretation.")
    answer = dspy.OutputField(desc="Recommendation: Buy, Hold, or Sell.")
    rationale = dspy.OutputField(desc="Reasoning behind the recommendation.")

# Define the DSPy module with Chain of Thought
class PortfolioCoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(PortfolioRecommendation)

    def forward(self, headline, fundamentals, market_conditions):
        return self.prog(headline=headline, fundamentals=fundamentals, market_conditions=market_conditions)

# Define the evaluation metric
def portfolio_metric(example, pred, trace=None):
    return example.answer.lower() == pred.answer.lower()

# dspy.MIPROv2(   metric: Callable, 
#                 prompt_model: Any | None = None, 
#                 task_model: Any | None = None, 
#                 teacher_settings: dict | None = None, 
#                 max_bootstrapped_demos: int = 4, 
#                 max_labeled_demos: int = 4, 
#                 auto: Literal['light', 'medium', 'heavy'] | None = 'light', 
#                 num_candidates: int | None = None, 
#                 num_threads: int | None = None, 
#                 max_errors: int | None = None, 
#                 seed: int = 9, 
#                 init_temperature: float = 0.5, 
#                 verbose: bool = False, 
#                 track_stats: bool = True, 
#                 log_dir: str | None = None, 
#                 metric_threshold: float | None = None)

# Initialize and compile the module with MIPROv2
config = dict(
    max_bootstrapped_demos=6,
    max_labeled_demos=6,
    metric=portfolio_metric
)
optimizer = MIPROv2(**config)
compiled_cot = optimizer.compile(
    PortfolioCoT(),
    trainset=trainset,
    valset=devset,
    requires_permission_to_run=False  # Set to True if running in restricted environment
)

# Save the optimized module
compiled_cot.save('portfolio_cot.json')

# Set up DSPy evaluator
evaluator = Evaluate(
    devset=devset,
    metric=portfolio_metric,
    num_threads=4,
    display_progress=True,
    display_table=5  # Show up to 5 example predictions in a table
)

# Evaluate the compiled module
eval_results = evaluator(compiled_cot)
print(f"Evaluation Accuracy: {eval_results:.2%}")

2025/07/19 04:46:18 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: False
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 10

2025/07/19 04:46:18 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/07/19 04:46:18 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/07/19 04:46:18 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


 30%|███       | 6/20 [00:02<00:06,  2.19it/s]


Bootstrapped 6 full traces after 6 examples for up to 1 rounds, amounting to 6 attempts.
Bootstrapping set 4/6


 30%|███       | 6/20 [00:02<00:05,  2.66it/s]


...

2025/07/19 04:48:38 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 7.00 / 8 (87.5%): 100%|██████████| 10/10 [00:10<00:00,  1.02s/it]

2025/07/19 04:48:39 INFO dspy.evaluate.evaluate: Average Metric: 7.0 / 10 (70.0%)


,headline,fundamentals,market_conditions,example_answer,example_rationale,reasoning,pred_answer,pred_rationale,portfolio_metric,answer,rationale
0,"TechCorp unveils new AI chip, analysts optimistic.","P/E: 23, Rev Growth: 14%",VIX: 17 (low),Buy,Innovative product and strong fundamentals in stable market.,"The unveiling of the new AI chip by TechCorp, coupled with optimis...",Buy,"The unveiling of the new AI chip, strong revenue growth, and a low...",✔️ [True],NaN,NaN
1,PharmaCo faces lawsuit over drug side effects.,"P/E: 28, Rev Growth: -4%",VIX: 24 (high),NaN,NaN,NaN,NaN,NaN,,Sell,Legal risks and negative growth in volatile market.
2,"EnergyCo reports steady output, no new contracts.","P/E: 19, Rev Growth: 5%",VIX: 20 (moderate),Hold,Stable performance but no strong growth catalyst.,The company's steady output but lack of new contracts may indicate...,Hold,Given the stable but unremarkable performance and moderate market ...,✔️ [True],NaN,NaN
3,"RetailCo launches loyalty program, mixed reviews.","P/E: 21, Rev Growth: 6%",VIX: 19 (moderate),Hold,"Loyalty program impact unclear, moderate market conditions.",The launch of a loyalty program by RetailCo could potentially boos...,Hold,"While the loyalty program has potential benefits, the mixed review...",✔️ [True],NaN,NaN
4,AutoCo ramps up EV production with new factory.,"P/E: 20, Rev Growth: 12%",VIX: 16 (low),NaN,NaN,NaN,NaN,NaN,,Buy,Increased production capacity in stable market is positive.


Evaluation Accuracy: 7000.00%


In [11]:
# Example usage: Predict portfolio recommendation for a new stock
new_headline = "TechCorp launches 5G infrastructure division, strong demand expected."
new_fundamentals = "P/E: 21, Rev Growth: 13%"
new_market_conditions = "VIX: 16 (low)"
prediction = compiled_cot(
    headline=new_headline,
    fundamentals=new_fundamentals,
    market_conditions=new_market_conditions
)
print(f"\nExample Prediction:")
print(f"Headline: {new_headline}")
print(f"Fundamentals: {new_fundamentals}")
print(f"Market Conditions: {new_market_conditions}")
print(f"Recommendation: {prediction.answer}")
print(f"Rationale: {prediction.rationale}")


Example Prediction:
Headline: TechCorp launches 5G infrastructure division, strong demand expected.
Fundamentals: P/E: 21, Rev Growth: 13%
Market Conditions: VIX: 16 (low)
Recommendation: Buy
Rationale: The combination of a promising new business division, solid growth prospects, and a stable market environment makes TechCorp an attractive investment opportunity, supporting a buy recommendation.


# Loadback & Use

In [ ]:
"""loaded_dspy_program = dspy.ChainOfThought("question -> answer") # Recreate the same program.
loaded_dspy_program.load("./dspy_program/program.json")

assert len(compiled_dspy_program.demos) == len(loaded_dspy_program.demos)"""

# Whole Program Saving & Load

"""compiled_dspy_program.save("./dspy_program/", save_program=True)
loaded_dspy_program = dspy.load("./dspy_program/")

assert len(compiled_dspy_program.demos) == len(loaded_dspy_program.demos)"""